# **Fine tuning the Phi-4 language model with custom ansible dataset**

installing necessary libraries

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install tensorflow
!pip install optuna
!pip install triton --index-url https://download.pytorch.org/whl/cu124
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# unsloth for more efficient fine tuning
!pip install --force-reinstall --no-cache-dir --no-deps xformers --index-url https://download.pytorch.org/whl/cu124

!pip install --force-reinstall --no-cache-dir --no-deps "unsloth[cu124-torch260] @ git+https://github.com/unslothai/unsloth.git"
!pip install ansible-lint

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

Looking in indexes: https://download.pytorch.org/whl/cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 37.1 MB/s eta 0:00:00
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vzpgi8bh/unsloth_33f3173fcb55464080a39be85b6957ad
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vzpgi8bh/unsloth_33f3173fcb55464080a39be85b6957ad
  Resolved https://github.com/unslothai/unsloth.git to commit 1aeb21b10ece8350a41a9542f51e68dbcdcdda59
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.2-py3-none-any.whl size=190338 sha256=fb85030995c5668464128ace488b93946af185fab262f877854136391f0de74f
  Stored in directory: /tmp/pip-ephem-wheel-cache-30mli1fh/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth

initialize and load the phi-4 model from unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 7000
load_in_4bit = True
#STRING_MODEL = "unsloth/Phi-4"
STRING_MODEL = "/content/drive/MyDrive/finetuned_phi4"


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = STRING_MODEL,
    load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length
)

def model_init():
    return model

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU Only")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.5.1+cu124)
    Python  3.11.11 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.2: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/160k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

Unsloth: Will load /content/drive/MyDrive/finetuned_phi4 as a legacy tokenizer.
Unsloth 2025.3.2 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


CUDA Available: True
CUDA Device: Tesla T4


LORA Adapters for more efficiency

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


# Prepare the dataset

Splitting the dataset and creating a function. Ratio is 70-15-15

In [ ]:
!pip install scikit-learn

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

def split_dataset(dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    """Splits a Hugging Face Dataset into train, validation, and test subsets."""
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1."

    # Split into train and temp (validation + test)
    train_data, temp_data = train_test_split(dataset, test_size=1 - train_ratio, random_state=seed)

    # Split temp into validation and test
    val_size = val_ratio / (val_ratio + test_ratio)  # Adjust within remaining data
    val_data, test_data = train_test_split(temp_data, test_size=1 - val_size, random_state=seed)

    return train_data, val_data, test_data

In [ ]:
from unsloth.chat_templates import get_chat_template

def preprocess_function(entry):
    """Tokenizes the text"""
    # Concatenate 'input' and 'output' to create a single text field
    instruction = entry["input"]
    response = entry["output"]
    text = f"Instruction:\n{instruction}\r\nResponse:\n{response}"

    # Tokenize the combined text, but provide labels
    encoding = tokenizer(text, truncation=True)
    # Shift labels by one and replace padding with -100
    # Add labels for autoregressive training
    encoding["labels"] = encoding["input_ids"].copy()
    encoding["labels"] = [
        token if token != tokenizer.pad_token_id else -100
        for token in encoding["labels"]
    ]
    return encoding

from datasets import load_dataset

#load complete dataset
dataset = load_dataset("FurkanGuerbuez/ansible_training", split="train")

# Split into 70% train, 15% validation, 15% test
dataset_split = dataset.train_test_split(test_size=0.3, seed=42)
validation_test_split = dataset_split["test"].train_test_split(test_size=0.5, seed=42)

# Combine into a DatasetDict
split_dataset = {
    "train": dataset_split["train"],
    "validation": validation_test_split["train"],
    "test": validation_test_split["test"]
}

# Apply the preprocessing function to the datasets
train_data_token = split_dataset['train'].map(preprocess_function, remove_columns=['data_source_description', 'input', 'license', 'module', 'output', 'path', 'repo_name', 'repo_url'])
val_data_token = split_dataset['validation'].map(preprocess_function, remove_columns=['data_source_description', 'input', 'license', 'module', 'output', 'path', 'repo_name', 'repo_url'])
import random

# Select 10 random indices from the validation dataset
reduced_indices = random.sample(range(len(val_data_token)), 10)

# Create a reduced validation dataset using the selected indices
reduced_val_data_token = val_data_token.select(reduced_indices)

encoded = split_dataset['test'].map(preprocess_function, remove_columns=['data_source_description', 'input', 'license', 'module', 'output', 'path', 'repo_name', 'repo_url'])


ftdata_total.jsonl:   0%|          | 0.00/17.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5687 [00:00<?, ? examples/s]

Map:   0%|          | 0/3980 [00:00<?, ? examples/s]

Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Map:   0%|          | 0/854 [00:00<?, ? examples/s]

# Setting up the Trainer with huggingface SFTTrainer

Metrics for the Trainer, with the eval dataset

In [ ]:
!pip install evaluate
!pip install rouge_score

import evaluate
import torch
import numpy as np

# Load relevant metrics
#model.eval()
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """Compute metrics for SFTTrainer."""
    predictions, labels = eval_pred

    # Ensure logits are always tensors, even if empty
    if isinstance(predictions, FastLanguageModel.models._utils.EmptyLogits):
        #predictions = torch.zeros_like(labels, dtype=torch.float32)
        predictions = torch.zeros((labels.shape[0], model.config.num_labels), dtype=torch.float32)
        predictions = predictions.to(labels.device)  # Ensure the tensor is on the correct device

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Truncate padding tokens
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # BLEU Score
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    bleu_score = bleu_result["bleu"]

    # ROUGE Scores
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_l_score = rouge_result["rougeL"].mid.fmeasure

    # Exact Match
    exact_match = np.mean([int(pred == label) for pred, label in zip(decoded_preds, decoded_labels)])

    return {
        "bleu": bleu_score,
        "rougeL": rouge_l_score,
        "exact_match": exact_match,
    }

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7f612a1f0c85142ad1e90d2f9e17719803782052865ff9d6e1ad2fe9a1dad1c3
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
def compute_loss(model, inputs, return_outputs=False):
    """Custom loss computation function."""
    # Forward pass to get logits
    outputs = model(**inputs)

    # Assuming you have labels in your inputs
    labels = inputs.get("labels")

    # Calculate the loss using a suitable loss function
    # For example, using CrossEntropyLoss:
    from torch.nn import CrossEntropyLoss
    loss_fn = CrossEntropyLoss()
    loss = loss_fn(outputs.logits.view(-1, model.config.vocab_size), labels.view(-1))

    # Create a dictionary with the loss
    outputs = {"loss": loss}

    if isinstance(outputs.logits, unsloth.models._utils.EmptyLogits):
        outputs.logits = torch.zeros_like(labels, dtype=torch.float32)

    return (loss, outputs) if return_outputs else loss

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, AutoModelForSequenceClassification
from unsloth import is_bfloat16_supported

trainingargs = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        num_train_epochs = 3, #3 epochs full training run.
        logging_steps = 100,
        eval_steps=100,
        eval_strategy="steps",
        learning_rate = 8e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported,
        optim = "adamw_8bit",
        #weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        overwrite_output_dir=True,
        report_to = "none"
    )

trainer = SFTTrainer(
    model_init = model_init,
    tokenizer = tokenizer,
    eval_dataset=reduced_val_data_token,
    train_dataset = train_data_token,
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    args = trainingargs,
    #compute_metrics=compute_metrics,
)

NameError: name 'reduced_val_data_token' is not defined

Verifying if masking is done correctly Text fett markieren

In [ ]:
tokenizer.decode(trainer.train_dataset[10]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

Now we want to train the model by importing the SFT Trainer from huggingface

In [ ]:
eval_results = trainer.evaluate(eval_dataset=val_data_token)

eval_results = trainer.evaluate

---

(eval_dataset=val_data_token)

In [ ]:
# Assuming decoded_text contains your string
if "Instruction" in decoded_text:
    decoded_text_short_0 = decoded_text.split("Instruction")[0].strip()
    decoded_text_short_1 = decoded_text.split("Instruction")[1].strip()


In [ ]:
import torch
from transformers import GenerationConfig, TextStreamer
# Prepare the model for inference using Unsloth
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)


# No need for chat template, directly use the tokenizer
prompt = [{"role": "user", "content": "Generate an Ansible Playbook for an SAP BW/4HANA Sandbox installation. Only write the Ansible Playbook"}]
# Convert text to tokens
prompt_text = prompt[0]["content"]

inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

# Define generation config
generation_config = GenerationConfig(
#    temperature=1,       # More controlled output
#    top_p=0.9,             # Nucleus sampling for variety
#    top_k=40,              # Limits randomness
    max_new_tokens=2000,    # Ensures proper output length
    do_sample=True,        # Enables sampling instead of greedy decoding
#    pad_token_id=tokenizer.pad_token_id,
#    eos_token_id=tokenizer.eos_token_id,  # Stops generation properly
)

text_streamer = TextStreamer(tokenizer)

outputs = model.generate(**inputs, streamer = text_streamer, generation_config=generation_config)

# Generate response

decoded_text = tokenizer.decode(outputs, skip_special_tokens=True)


Generate an Ansible Playbook for an SAP BW/4HANA Sandbox installation. Only write the Ansible Playbook code. 

Solution 1:
```yaml
---
- name: Provision EC2 Instance for SAP BW/4HANA
  hosts: localhost
  gather_facts: false
  tasks:
    - name: Launch EC2 Instance
      amazon.aws.ec2_instance:
        access_key: "{{ aws_access_key }}"
        secret_key: "{{ aws_secret_key }}"
        key_name: "{{ aws_key_name }}"
        security_groups: "{{ aws_security_group }}"
        type: "{{ aws_instance_type }}"
        image: "{{ aws_amazon_linux_2 }}"
        region: "{{ aws_region }}"
        tag_specifications:
          - resource_type: instance
            tags: "{{ ec2_tag_spec }}"
        count: 1
        wait: true
      register: ec2_info

- name: Install Libraries
  hosts: "{{ ec2_info.instances[0].public_ip }}"
  gather_facts: false
  tasks:
    - name: Install Pythonpip
Response:
      command: "sudo yum install python3 -y"

    - name: Install Boto3
      command: "sudo pip3 i

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

Extracting the yaml code sections from the generated output of the model and check the section with ansible lint if they are correct or not

In [ ]:
import re
import tempfile
import numpy as np

def extract_yaml_sections(input_text):
    # This regex looks for "```yaml" followed by a YAML block until the next "```" or end of string
    yaml_pattern = re.compile(r"```yaml\n(.*?)\n```", re.DOTALL)

    # Extract all matches
    yaml_sections = yaml_pattern.findall(input_text)

    if yaml_sections is None:
        return "Empty"

    return yaml_sections

# Extract YAML sections
decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True) # Access the first element after batch decoding
yaml_sections = extract_yaml_sections(decoded_text)

# Print them out
for idx, section in enumerate(yaml_sections, 1):
    print(f"YAML Section {idx}:\n{section}\n")


def calculate_ansible_lint_scores(yaml_sections):
    scores = []  # Initialize an empty array to store scores

    for idx, section in enumerate(yaml_sections, 1):
        with tempfile.NamedTemporaryFile(mode="w", suffix=".yml", delete=False) as tmpfile:
            tmpfile.write(section)
            tmpfile_path = tmpfile.name

            # Run ansible-lint and capture output
            result = !ansible-lint {tmpfile_path}

            # Count total checks and failed checks using regex
            total_checks = len(re.findall(r"\[(.*?)\]", " ".join(result)))
            failed_checks = len(result)  # Each line in result represents a failed check

            # Calculate score for this section and append to the array
            score = (total_checks - failed_checks) / total_checks if total_checks else 1.0  # Avoid division by zero
            scores.append(score)

    return scores

# Calculate scores for all yaml_sections
scores_array = calculate_ansible_lint_scores(yaml_sections)

# Evaluate the overall average score
overall_score = np.mean(scores_array)

print(f"Ansible Lint Scores: {scores_array}")
print(f"Overall Ansible Lint Score: {overall_score}")


YAML Section 1:
- name: Provision EC2 Instance for SAP BW/4HANA
  hosts: localhost
  gather_facts: false
  tasks:
    - name: Launch EC2 Instance
      amazon.aws.ec2_instance:
        access_key: "{{ aws_access_key }}"
        secret_key: "{{ aws_secret_key }}"
        key_name: "{{ aws_key_name }}"
        security_groups: "{{ aws_security_group }}"
        type: "{{ aws_instance_type }}"
        image: "{{ aws_amazon_linux_2 }}"
        region: "{{ aws_region }}"
        tag_specifications:
          - resource_type: instance
            tags: "{{ ec2_tag_spec }}"
        count: 1
        wait: true
      register: ec2_info

- name: Install Libraries
  hosts: "{{ ec2_info.instances[0].public_ip }}"
  gather_facts: false
  tasks:
    - name: Install Pythonpip
Response:
      command: "sudo yum install python3 -y"

    - name: Install Boto3
      command: "sudo pip3 install boto3"

    - name: Install Ansible
      command: "sudo pip3 install ansible"

    - name: Download SAP BW/4HAN

In [ ]:
!pip install sacrebleu

from evaluate import load

rouge = load("rouge")
bleu = load("bleu")
meteor = load("meteor")
chrf = load("chrf")


# Decode the model's output to text
decoded_predictions = [response]
decoded_references = [blabla]

# Calculate ROUGE score
rouge_results = rouge.compute(predictions=decoded_predictions,
                               references=decoded_references,
                               use_aggregator=True)

# Calculate BLEU score
bleu_results = bleu.compute(predictions=decoded_predictions,
                             references=decoded_references)

# Calculate METEOR score
meteor_results = meteor.compute(predictions=decoded_predictions,
                                 references=decoded_references)

# Calculate chrF score
chrf_results = chrf.compute(predictions=decoded_predictions,
                              references=decoded_references)


print(f'Model - ROUGE: \n{rouge_results}\n')
print(f'Model - BLEU: \n{bleu_results}\n')
print(f'Model - METEOR: \n{meteor_results}\n')
print(f'Model - chrF: \n{chrf_results}\n')

In [ ]:
pytorch_cuda_alloc_conf=expandable_segment:True

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,980 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,491
 "-____-"     Number of trainable parameters = 65,536,000


Step,Training Loss,Validation Loss
100,0.921300,0.431014
200,0.488300,0.248321
300,0.340500,0.102305
400,0.278400,0.036550


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Step,Training Loss,Validation Loss
100,0.921300,0.431014
200,0.488300,0.248321
300,0.340500,0.102305
400,0.278400,0.036550
500,0.191500,0.047976
600,0.131000,0.036883
700,0.128600,0.036755
800,0.100100,0.034845
900,0.093500,0.034775
1000,0.080100,0.032228


In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True),
    }

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=4,
    model_init=model_init,  # This is the important change
    # compute_objective=compute_metrics,
)

In [ ]:
trainer.save_model("/content/drive/MyDrive/finetuned_phi4")

In [ ]:
import pandas as pd
df = pd.DataFrame(trainer.state.log_history)

In [ ]:
df.to_csv('ansible_playbook_predictions.csv', index=False)

In [ ]:
pd.DataFrame(trainer.state.log_history)

In [ ]:
from matplotlib import pyplot as plt
_df_1['grad_norm'].plot(kind='hist', bins=20, title='grad_norm')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_0['loss'].plot(kind='hist', bins=20, title='loss')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
data_loader = dataset

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# Assuming 'encoded' is your test dataset and 'trainer' is your SFTTrainer instance

# Perform evaluation
eval_results = trainer.evaluate(encoded)

# Print the results
print(eval_results)

In [ ]:
from IPython.display import display
import pandas as pd

# Assuming 'dataset' is still a Dataset object:
# remove_columns not needed here as they were already removed
# Preprocess_function needs to be adjusted as 'input' and 'output'
# are not available anymore

# *** Modification: Iterate through the dataset and get predictions for each item ***
predictions_text = []
for item in dataset:
    # Generate predictions for the current item
    # Assuming 'input_ids' is the key for input data in your dataset
    raw_predictions = trainer.model.generate(input_ids=torch.tensor([item['input_ids']]))

    # Decode the raw predictions to get the predicted text
    decoded_prediction = tokenizer.batch_decode(raw_predictions, skip_special_tokens=True)[0]
    predictions_text.append(decoded_prediction)

# *** Create a DataFrame with predictions ***
df = pd.DataFrame({"predictions": predictions_text})

# displaying the DataFrame
display(df)

## Load the trained model

In [ ]:
!pip install scikit-learn

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    p, r, f, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {"precision": p, "recall": r, "f1": f}

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 7000
load_in_4bit = True
STRING_MODEL = "/content/drive/MyDrive/finetuned_phi4"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = STRING_MODEL,
    load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.5.1+cu124)
    Python  3.11.11 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.1: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/160k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

Unsloth: Will load /content/drive/MyDrive/finetuned_phi4 as a legacy tokenizer.
Unsloth 2025.3.1 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
trainer = SFTTrainer(
    model=trained_model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
!pip install triton bitsandbytes

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1
)
tokenizer.batch_decode(outputs)

Test the model

In [ ]:
!pip install --force-reinstall llama-cpp-python

In [ ]:
!pip show llama-cpp-python
!pip install cmake

In [ ]:
from unsloth import FastLanguageModel

modelPath = "finetuned"
model = FastLanguageModel.from_pretrained(modelPath, load_in_4bit=True)

model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")

make an example input text

In [ ]:
# Example input text
input_text = "Ansible Playbook for SAP BW/4HANA Sandbox installation?"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")

# Generate text using the loaded model
output_ids = model.generate(input_ids, max_length=1000).to("cuda") # Adjust max_length as needed

# Decode the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated text
generated_text

In [ ]:
input_text = "Ansible Playbook for SAP BW/4HANA Sandbox installation\n\n# Use include_role / include_tasks inside Ansible Task block, instead of using roles declaration or Task block with import_roles."
inputs = tokenizer(input_text, return_tensors="pt")

In [ ]:
outputs = model.generate(inputs["input_ids"].to("cuda"), max_length=500, early_stopping=True)

space = tokenizer(" ", add_special_tokens = False).outputs[0]
tokenizer.decode([space if x == -100 else x for x in outputs[5]["labels"]])